In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.models as models
import cv2
import sys
import math
import random
import splitfolders
import torchsummary
from tqdm.auto import tqdm
from ResNet_18 import resnet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':1e-2,
    'BATCH_SIZE':24,
    'SEED':42
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [6]:
user_list = ['user01', 'user02', 'user03', 'user04',
             'user05', 'user06', 'user07', 'user08',
             'user09', 'user10', 'user11', 'user12',
             'user21', 'user22', 'user23', 'user24',
             'user25', 'user26', 'user27', 'user28',
             'user29', 'user30']

label_path_list = ['/user01-06/', '/user07-10/', '/user11-12/', '/user21-25/', '/user26-30/']
label_path = ''

for file in user_list:
    if file in user_list[:6]:
        label_path = label_path_list[0]
    elif file in user_list[6:10]:
        label_path = label_path_list[1]
    elif file in user_list[10:12]:
        label_path = label_path_list[2]
    elif file in user_list[12:17]:
        label_path = label_path_list[3]
    else:
        label_path = label_path_list[4]

In [24]:
mGps_df = pd.DataFrame([], columns=['ts', 'user','lat','lon'])
mGps_df

,ts,user,lat,lon


In [25]:
# Read Data
dataset_path = './userdata/'
path_list = os.listdir(dataset_path)
for file in path_list:
    file_path = os.path.join(dataset_path, file)
    user_list = os.listdir(file_path)
    for user in user_list:
        data_path = os.path.join(file_path, user)
        dir_list = os.listdir(data_path)
        
        mGps_data = []
        
        for item in dir_list:
#             csv_file = data_path + "/" + item + "/" + item + "_label.csv"
            sub_path = data_path + "/" + item + "/" + "mGps"
            ts_list = os.listdir(sub_path)
            for ts in ts_list:
                pt = os.path.join(sub_path, ts)
                rdr = pd.read_csv(pt)
                ts = int(ts[:-4])
                mean_lat = rdr.mean()['lat']
                mean_lon = rdr.mean()['lon']
                mGps_df.loc[len(mGps_df)] = [ts, user, mean_lat, mean_lon]
    
#     user_all_data = []
#     user_path = os.path.join(label_path, user)
#     bts_list = os.listdir(user_path)

#     for bts in bts_list:
#         if bts.endswith('.csv'):
#             continue
#         bts_path = os.path.join(user_path, bts)
#         e4Eda_path = os.path.join(bts_path, 'e4Eda')
#         e4Temp_path = os.path.join(bts_path, 'e4Temp')

#         e4Eda_list = os.listdir(e4Eda_path)
#         e4Temp_list = os.listdir(e4Temp_path)

#         tmp_ts
#         ts_df = pd.DataFrame([], columns=['timestamp'])
#         ts_df['timestamp'] = tmp_ts
#         ts_df

#         # bts 폴더 내에서 eda_temp 없으면 만들어줌
#         if not os.path.exists(bts_path + '/eda_temp'):
#             os.makedirs(bts_path + '/eda_temp')

#         for sts in e4Eda_list:
#             # sts가 Eda, Temp 둘다 있는거만
#             if sts not in e4Temp_list:
#                 continue

#             # timestamp기준으로 eda merge
#             sts_eda_path = os.path.join(e4Eda_path, sts)
#             sts_eda_df = pd.read_csv(sts_eda_path)
#             eda_df = pd.merge(ts_df, sts_eda_df, on = 'timestamp', how = 'left')


#             eda_temp_path = os.path.join(bts_path, 'eda_temp', sts)

#             # 해당 csv 존재하면 넘어감
#             if os.path.exists(eda_temp_path):
#                 continue
#             else:
#                 # csv 생성
#                 eda_temp_df.to_csv(eda_temp_path)





    #     bts_csvfile = os.path.join(bts_path, bts + '_label.csv')
    #     df = pd.read_csv(bts_csvfile)
    #     user_all_data.append(df)
    # df_all = pd.concat(user_all_data, axis=0)
    # df_all = df_all.drop(columns=['actionSub', 'actionSubOption', 'condition', 'conditionSub1Option', 'conditionSub2Option'])
    # df_all.to_csv(os.path.join(user_path, user + '_label.csv'))

ValueError: invalid literal for int() with base 10: '.~lock.1599757200.'

In [26]:
mGps_df

,ts,user,lat,lon
0,1601079420,user12,37.614050,126.927204
1,1601124420,user12,37.614073,126.927211
2,1601123400,user12,37.614123,126.927267
3,1601131440,user12,37.614254,126.927210
4,1601089680,user12,37.614076,126.927215
...,...,...,...,...
115651,1599803040,user06,37.498337,127.034569
115652,1599811140,user06,37.498340,127.034578
115653,1599789780,user06,37.498335,127.034569
115654,1599823980,user06,37.513588,127.045812


In [27]:
mGps_df.to_csv('mGps_data.csv')